In [47]:
import sys
!{sys.executable} -m pip install --upgrade --pre itk imageio pynrrd
!{sys.executable} -m pip install numcodecs zarr xarray

Requirement already up-to-date: itk in /home/matt/bin/miniconda3/envs/nwbwidgets/lib/python3.7/site-packages (5.1.0)
Requirement already up-to-date: imageio in /home/matt/bin/miniconda3/envs/nwbwidgets/lib/python3.7/site-packages (2.8.0)
Requirement already up-to-date: pynrrd in /home/matt/bin/miniconda3/envs/nwbwidgets/lib/python3.7/site-packages (0.4.2)
Processing /home/matt/.cache/pip/wheels/e1/5b/25/24c685604b91139aba00a5b6299b53e7a0661f737f27782559/zarr-2.4.0-cp37-none-any.whl
Processing /home/matt/.cache/pip/wheels/1d/d9/58/9808b306744df0208fccc640d3d9952a5bc7468502d42897d5/asciitree-0.3.3-cp37-none-any.whl
  Using cached fasteners-0.15-py2.py3-none-any.whl (23 kB)
  Using cached monotonic-1.5-py2.py3-none-any.whl (5.3 kB)


In [1]:
import json
import imageio
import nrrd
import numpy as np
import xarray as xr
import numpy as np
from numcodecs import Blosc, blosc
import zarr

import itk
from itkwidgets import view

Based on: https://github.com/BICCN/cell-locator/blob/ac84ce35e5cecdacd063009ce955bbd8cc489a11/Modules/Scripted/Home/CMakeLists.txt


In [2]:
!wget http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/annotation/ccf_2017/annotation_50.nrrd

--2020-04-30 23:15:45--  http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/annotation/ccf_2017/annotation_50.nrrd
Resolving download.alleninstitute.org (download.alleninstitute.org)... 63.237.233.56
Connecting to download.alleninstitute.org (download.alleninstitute.org)|63.237.233.56|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 880845 (860K)
Saving to: ‘annotation_50.nrrd.1’

annotation_50.nrrd. 100%[===================>] 860.20K   427KB/s    in 2.0s    

2020-04-30 23:15:47 (427 KB/s) - ‘annotation_50.nrrd.1’ saved [880845/880845]



In [3]:
!wget -O ontology-original.json "http://api.brain-map.org/api/v2/data/Structure/query.json?criteria=%5Bgraph_id$eq1%5D&num_rows=2000"

--2020-04-30 23:15:49--  http://api.brain-map.org/api/v2/data/Structure/query.json?criteria=%5Bgraph_id%5D&num_rows=2000
Resolving api.brain-map.org (api.brain-map.org)... 63.237.233.29
Connecting to api.brain-map.org (api.brain-map.org)|63.237.233.29|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘ontology-original.json’

ontology-original.j     [         <=>        ]   1.01M   475KB/s    in 2.2s    

2020-04-30 23:15:57 (475 KB/s) - ‘ontology-original.json’ saved [1056857]



In [4]:
!python -m json.tool ontology-original.json ontology-formatted.json

In [2]:
def load_mapping(file_path):
    with open(file_path) as content:
        mapping = json.load(content)
        return {int(key): int(value) for (key, value) in mapping.items()}

In [3]:
annotation = nrrd.read('annotation_50.nrrd')

In [4]:
array = annotation[0]
array.shape

(264, 160, 228)

In [5]:
flat = array.flatten()

In [6]:
labels = np.unique(flat)
labels

array([        0,         1,         2,         6,         7,         9,
              10,        12,        15,        17,        19,        20,
              23,        26,        27,        28,        30,        33,
              35,        36,        38,        41,        42,        50,
              52,        54,        56,        58,        59,        62,
              63,        64,        66,        67,        68,        72,
              74,        75,        78,        81,        83,        84,
              88,        91,        93,        96,        97,        98,
             100,       101,       102,       105,       106,       108,
             113,       114,       115,       117,       118,       120,
             121,       122,       123,       125,       126,       128,
             129,       131,       132,       133,       136,       139,
             140,       143,       145,       146,       147,       148,
             149,       153,       155,       156, 

In [7]:
a2i_mapping = {int(value): idx for idx, value in enumerate(labels)}

In [8]:
with open('allen_id_to_label.json', 'w') as fileContents:
    fileContents.write(json.dumps(a2i_mapping, sort_keys=True, indent=2))

In [9]:
i2a_mapping = {idx: int(value) for idx, value in enumerate(labels)}

In [10]:
with open('label_to_allen_id.json', 'w') as fileContents:
    fileContents.write(json.dumps(i2a_mapping, sort_keys=True, indent=2))

In [11]:
for idx, value in enumerate(labels):
    if value == idx:
      continue
    #print(idx, value)
    array[array == value] = idx

In [12]:
nrrd.write('annotation_50_contiguous.nrrd', annotation[0].astype(np.uint16), annotation[1])

In [13]:
label_count = annotation[0].max()
label_count

670

In [14]:
with open('ontology-formatted.json') as fp:
    json_data = json.load(fp)
json_data

{'success': True,
 'id': 0,
 'start_row': 0,
 'num_rows': 2000,
 'total_rows': 13708,
 'msg': [{'acronym': 'r1BMp',
   'atlas_id': None,
   'color_hex_triplet': 'D654E6',
   'depth': 8,
   'failed': False,
   'failed_facet': 734881840,
   'graph_id': 17,
   'graph_order': 1530,
   'hemisphere_id': 3,
   'id': 16997,
   'name': 'periventricular stratum of r1BM',
   'neuro_name_structure_id': None,
   'neuro_name_structure_id_path': None,
   'ontology_id': 12,
   'parent_structure_id': 16996,
   'safe_name': 'periventricular stratum of r1BM',
   'sphinx_id': 11447,
   'st_level': 10,
   'structure_id_path': '/15564/15565/16808/16809/16915/16967/16994/16996/16997/',
   'structure_name_facet': 3761647284,
   'weight': 8390},
  {'acronym': 'DLG',
   'atlas_id': None,
   'color_hex_triplet': '8C4685',
   'depth': 5,
   'failed': False,
   'failed_facet': 734881840,
   'graph_id': 8,
   'graph_order': 254,
   'hemisphere_id': 3,
   'id': 128013074,
   'name': 'dorsal lateral geniculate nucleu

In [15]:
num_rows = json_data['num_rows']
num_rows

2000

In [16]:
allen_categorical_colors = [None]*label_count
allen_ontology = [None]*label_count

In [17]:
def hex_to_rgb(_hex):
  _hex = _hex.lstrip('#')
  hlen = len(_hex)
  return [float(int(_hex[i:i+int(hlen/3)], 16))/255. for i in range(0, hlen, int(hlen/3))]

In [18]:
for row in range(num_rows):
    onto_id = int(json_data['msg'][row]['id'])
    if not onto_id or onto_id < 0:
        continue
    if not onto_id in a2i_mapping:
        continue
    label = a2i_mapping[onto_id]
    info = json_data['msg'][row]
    try:
          name = info['safe_name']
    except KeyError:
          name = info['name']
    allen_ontology[label] = { 'acronym': info['acronym'],
                            'name': name,
                            'id': info['id']
                            }
    hex_color = info['color_hex_triplet']
    color = hex_to_rgb(hex_color)
    allen_categorical_colors[label] = color

In [19]:
allen_ontology

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 {'acronym': 'SCiw',
  'name': 'Superior colliculus motor related intermediate white layer',
  'id': 17},
 None,
 {'acronym': 'ENTl2',
  'name': 'Entorhinal area lateral part layer 2',
  'id': 20},
 None,
 {'acronym': 'SCdg',
  'name': 'Superior colliculus motor related deep gray layer',
  'id': 26},
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 {'acronym': 'AD', 'name': 'Anterodorsal nucleus', 'id': 64},
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 {'acronym': 'PVi',
  'name': 'Periventricular hypothalamic nucleus intermediate part',
  'id': 118},
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 {'acronym': 'V3', 'name': 'third ventricle', 'id': 129},
 None,
 None,
 None,
 None,
 None,
 {'acronym': 'AQ', 'name': '

In [20]:
allen_categorical_colors

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 [1.0, 0.5647058823529412, 1.0],
 None,
 [0.19607843137254902, 0.7215686274509804, 0.1450980392156863],
 None,
 [1.0, 0.5647058823529412, 1.0],
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 [1.0, 0.5647058823529412, 0.6235294117647059],
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 [1.0, 0.36470588235294116, 0.3137254901960784],
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 [0.6666666666666666, 0.6666666666666666, 0.6666666666666666],
 None,
 None,
 None,
 None,
 None,
 [0.6666666666666666, 0.6666666666666666, 0.6666666666666666],
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 [0.00392156862745098, 0.5764705882352941, 0.6],
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 No

In [21]:
allen_categorical_colors = [[0.0, 0.0, 0.0] if c is None else c for c in allen_categorical_colors]

In [22]:
allen_categorical_colors

[[0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [1.0, 0.5647058823529412, 1.0],
 [0.0, 0.0, 0.0],
 [0.19607843137254902, 0.7215686274509804, 0.1450980392156863],
 [0.0, 0.0, 0.0],
 [1.0, 0.5647058823529412, 1.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [1.0, 0.5647058823529412, 0.6235294117647059],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 

In [33]:
with open('allen_ontology_50.json', 'w') as fp:
    fp.write(json.dumps(allen_ontology, sort_keys=True, indent=2))
with open('allen_categorical_colors_50.json', 'w') as fp:
    fp.write(json.dumps(allen_categorical_colors))

In [24]:
image = itk.imread('annotation_50_contiguous.nrrd')

In [25]:
itk.imwrite(image, 'annotation_50_contiguous_itk.nrrd', compression=True)

In [26]:
image_da = itk.xarray_from_image(image)
image_da

<xarray.DataArray (z: 228, y: 160, x: 264)>
array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=uint16)
Coordinates:
  * x        (x) float64 0.0 50.0 100.0 150.0 ... 1.305e+04 1.31e+04 1.315e+04
  * y        (y) float64 0.0 50.0 100.0 150.0 ... 7.85e+03 7.9e+03 7.95e+03
  * z        (z) float64 0.0 50.0 100.0 150.0 ... 1.125e+04 1.13e+04 1.135e+04
Attributes:
    direction:  [[1. 0. 0.]\n [0. 1. 0.]\n [0. 0. 1.]]

In [27]:
dataset_name = 'allen_ccfv3_annotation'
store_name = 'allen_ccfv3_annotation_50_contiguous.zarr'

In [28]:
units = 'μm'
image_da.attrs['units'] = units
image_da.attrs

{'direction': array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 'units': 'μm'}

In [29]:
image_ds = image_da.to_dataset(name=dataset_name)
image_ds

<xarray.Dataset>
Dimensions:                 (x: 264, y: 160, z: 228)
Coordinates:
  * x                       (x) float64 0.0 50.0 100.0 ... 1.31e+04 1.315e+04
  * y                       (y) float64 0.0 50.0 100.0 ... 7.9e+03 7.95e+03
  * z                       (z) float64 0.0 50.0 100.0 ... 1.13e+04 1.135e+04
Data variables:
    allen_ccfv3_annotation  (z, y, x) uint16 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0

In [30]:
store = zarr.DirectoryStore(store_name)

# NOSHUFFLE since we will be visualizing with WebAssembly, which does not currently have support for intrinsics
blosc.use_threads = False
# Crashing (?)
compressor = Blosc(cname='zstd', clevel=5, shuffle=Blosc.NOSHUFFLE)
#compressor = Blosc(clevel=5, shuffle=Blosc.NOSHUFFLE)
chunk_size = 64

In [31]:
image_ds.to_zarr(store,
                 mode='w',
                 compute=True,
                 encoding={dataset_name: {'chunks': [chunk_size]*3, 'compressor': compressor}})

In [32]:
# After all modifications to the store are complete, consolidate the metadata so it is 'cloud-ready'.
zarr.consolidate_metadata(store)

<zarr.hierarchy.Group '/'>